### Python Debugging with __`pdb`__
* __`pdb`__ can be run
  * from the commandline
  * from within the Python interpreter
  * from within your Python program
  * from within Jupyter!
* important to distinguish between using __`pdb`__ to run or step through a program, vs. using __`pdb`__ in post-mortem mode, to determine why a program crashed
  * we will demonstrate both...

### Let's first run from within Jupyter...
* In order to do that, it's helpful to be familiar with a couple of Jupyter's "magic" commands
   * __`%debug`__ = start __`pdb`__ after a crash
   * __`%pdb`__ = toggle automatic calling of __`pdb`__ after a crash
   * __`%run`__ = run a named file inside of Jupyter
     * -d = run your program under the control of __`pdb`__
     * -t = output approximate timing information
     * -N__n__ (used with -t) = run the program __n__ times

### First we'll create a crash and do a post-mortem debug...
* Here's some code which computes prime numbers using the Sieve of Eratosthenes
* Using random(), I've hard-coded a bug which occurs 0.5% of the time, causing a crash


In [1]:
%pdb

Automatic pdb calling has been turned ON


In [2]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [6]:
from random import random
limit = 100
numbers = list(range(2, limit + 1))
primes = []

while numbers:
    candidate = numbers[0]
    primes.append(candidate)
    for num in range(candidate, limit + 1, candidate):
        if num in numbers:
            rval = random()
            if rval > 0.99:
                raise ValueError
            else:
                numbers.remove(num)

print(primes)

ValueError: 

> <ipython-input-6-f7f74f07a6e0>(13)<module>()
     11             rval = random()
     12             if rval > 0.99:
---> 13                 raise ValueError
     14             else:
     15                 numbers.remove(num)

ipdb> list
      8     primes.append(candidate)
      9     for num in range(candidate, limit + 1, candidate):
     10         if num in numbers:
     11             rval = random()
     12             if rval > 0.99:
---> 13                 raise ValueError
     14             else:
     15                 numbers.remove(num)
     16 
     17 print(primes)

ipdb> q


### Basic __`pdb`__ commands
 * __c(ontinue)__ = continue running until the next breakpoint
 * __s(tep)__ = step, i.e., move one line ahead–stops inside a called function
 * __n(ext)__ = next, i.e., move one line ahead–executes called functions at (nearly) full speed, only stopping at the next line in the current function
 * __l(ist)__ = list code
   * current line in the current frame is indicated by `->`
   * if an exception is being debugged, the line where the exception was originally raised or propagated is indicated by `>>`, if it differs from the current line
 * __b(reak) `func`__ = breakpoint inside function `func`
 * __b(reak) `num`__ = breakpoint at line number `num`
 * __t(break)__ = same as __b__, but breakpoint is removed after first hit
 * __cl(ear)__ = clear breakpoints
 * __unt(il) `num`__ = continue execution until a line number >= `num` is reached
 * __r(eturn)__ = continue execution until current function returns
 * __j(ump) `num`__ = jump back to line `n` (can't jump into a loop)
 * __p `expression`__ = print `expression`
 * __pp `expression`__ = pretty print `expression`
 * __q(uit)__ = quit

### Next, let's run an "outside" script directly from within Jupyter
* and then let's try running it with debugging turned on

In [9]:
pwd

'/Users/rick446/src/arborian-classes/src'

In [10]:
%%file data/primes.py
from random import random

limit = 100
numbers = list(range(2, limit + 1))
primes = []

while numbers:
    candidate = numbers[0]
    primes.append(candidate)
    for num in range(candidate, limit + 1, candidate):
        if num in numbers:
            numbers.remove(num)
print(primes)



Overwriting data/primes.py


In [11]:
%run data/primes.py

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [12]:
!python data/primes.py

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [13]:
# might have to restart the kernel before doing this
%run -d data/primes.py

Breakpoint 1 at /Users/rick446/src/arborian-classes/src/data/primes.py:1
NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> /Users/rick446/src/arborian-classes/src/data/primes.py(1)<module>()
1---> 1 from random import random
      2 
      3 limit = 100
      4 numbers = list(range(2, limit + 1))
      5 primes = []

ipdb> n
> /Users/rick446/src/arborian-classes/src/data/primes.py(3)<module>()
1     1 from random import random
      2 
----> 3 limit = 100
      4 numbers = list(range(2, limit + 1))
      5 primes = []

ipdb> n
> /Users/rick446/src/arborian-classes/src/data/primes.py(4)<module>()
      2 
      3 limit = 100
----> 4 numbers = list(range(2, limit + 1))
      5 primes = []
      6 

ipdb> n
> /Users/rick446/src/arborian-classes/src/data/primes.py(5)<module>()
      3 limit = 100
      4 numbers = list(range(2, limit + 1))
----> 5 primes = []
      6 
      7 while numbers:

ipdb> n
> /Users/rick446/src/arborian-classes/src/data/primes.py(7)<module>()
      5 p

ipdb> p num
2
ipdb> c
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


### To run from the command line, we need to jump out of the notebook for a moment...
* The command we'll use is

        python3 -m pdb script.py

### Now that we're back, we can run from the interpreter...
* This will allow us to test a module without having to do the save/run/import cycle

In [14]:
%%file data/mypdb.py
class PdbTester():
    def __init__(self, count):
        self.count = count

    def doit(self):
        for i in range(self.count):
            print(i)

Overwriting data/mypdb.py


In [15]:
cd data

/Users/rick446/src/arborian-classes/data


In [16]:
from mypdb import PdbTester
import pdb
pdb.run('PdbTester(3).doit()')

> <string>(1)<module>()
(Pdb) list
[EOF]
(Pdb) s
--Call--
> /Users/rick446/src/arborian-classes/data/mypdb.py(2)__init__()
-> def __init__(self, count):
(Pdb) list
  1  	class PdbTester():
  2  ->	    def __init__(self, count):
  3  	        self.count = count
  4  	
  5  	    def doit(self):
  6  	        for i in range(self.count):
  7  	            print(i)
[EOF]
(Pdb) n
> /Users/rick446/src/arborian-classes/data/mypdb.py(3)__init__()
-> self.count = count
(Pdb) 
--Return--
> /Users/rick446/src/arborian-classes/data/mypdb.py(3)__init__()->None
-> self.count = count
(Pdb) 
--Call--
> /Users/rick446/src/arborian-classes/data/mypdb.py(5)doit()
-> def doit(self):
(Pdb) 
> /Users/rick446/src/arborian-classes/data/mypdb.py(6)doit()
-> for i in range(self.count):
(Pdb) 
> /Users/rick446/src/arborian-classes/data/mypdb.py(7)doit()
-> print(i)
(Pdb) 
0
> /Users/rick446/src/arborian-classes/data/mypdb.py(6)doit()
-> for i in range(self.count):
(Pdb) c
1
2


### What if we have a program where the bug doesn't occur until the program has been running for a while?
* In a case such as this, we're going to want to invoke __`pdb`__ from *within* our program...
* ...but not from within the notebook, so we'll once again drop out of the notebook to examine the following code:

In [18]:
for num in range(1, 1000): # do something
    pass

print('do a lot more stuff')
x = 1
print('and now we get to the bug...')
if x == 1: # this is the buggy case
    import pdb
    pdb.set_trace() # causes the program to stop here, in the debugger
    print('buggy line')
    y = 1
else:
    y = x ** 2 # no debugging in this case
print(x)

do a lot more stuff
and now we get to the bug...
> <ipython-input-18-bff64036bdf8>(10)<module>()
-> print('buggy line')
(Pdb) list
  5  	x = 1
  6  	print('and now we get to the bug...')
  7  	if x == 1: # this is the buggy case
  8  	    import pdb
  9  	    pdb.set_trace() # causes the program to stop here, in the debugger
 10  ->	    print('buggy line')
 11  	    y = 1
 12  	else:
 13  	    y = x ** 2 # no debugging in this case
 14  	print(x)
[EOF]
(Pdb) j 11
> <ipython-input-18-bff64036bdf8>(11)<module>()
-> y = 1
(Pdb) c
1


In [19]:
for num in range(1, 1000): # do something
    pass

print('do a lot more stuff')
x = 1
print('and now we get to the bug...')
if x == 1: # this is the buggy case
    import ipdb
    ipdb.set_trace() # causes the program to stop here, in the debugger
    print('buggy line')
    y = 1
else:
    y = x ** 2 # no debugging in this case
print(x)

do a lot more stuff
and now we get to the bug...
> <ipython-input-19-b0947057f198>(10)<module>()
      9     ipdb.set_trace() # causes the program to stop here, in the debugger
---> 10     print('buggy line')
     11     y = 1

ipdb> c
buggy line
1


In [21]:
for num in range(1, 1000): # do something
    pass

print('do a lot more stuff')
x = 1
print('and now we get to the bug...')
if x == 1: # this is the buggy case
    breakpoint()  # Py3.7+
    print('buggy line')
    y = 1
else:
    y = x ** 2 # no debugging in this case
print(x)

do a lot more stuff
and now we get to the bug...
> <ipython-input-21-f8e9b9c679a3>(9)<module>()
-> print('buggy line')
(Pdb) c
buggy line
1


# Lab

Open [PDB Lab][pdb-lab]

[pdb-lab]: ./pdb-lab.ipynb